In [17]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
import datetime
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)


Conexion exitosa con la Base de datos:irvin_hisminsa


In [19]:
def consulta_sql(conn):
   
    gestante = conn.df(""" SELECT * FROM (
SELECT
    DISTINCT 
  nt.id_paciente
    , nt.anio
    , nt.mes
    , nt.id_establecimiento
    , mhe.nombre_eess
    , nt.anio_actual_paciente AS edad_anio
    , ROUND(
        (
            nt.fecha_atencion - mp.fecha_nacimiento
        )/ 30.44
        , 1
    ) AS edad_mes
    ,(
        nt.fecha_atencion - mp.fecha_nacimiento
    ) AS edad_dia
    , nt.fecha_atencion
    , nt.lote
    , nt.num_pag
    , nt.num_reg
    , nt2.codigo_item
    , nt2.hemoglobina
    , nt2.fecha_resultado_hb
    , nt2.id_centro_poblado
    , mhcp.descripcion_centro_poblado
    , mhcp.altitud_centro_poblado
    , nt.talla
    , nt.peso
    , nt.fecha_ultima_regla
    , nt.peso_pregestacional
    , mhtd.abrev_tipo_doc
    , mp.numero_documento
    , mp.fecha_nacimiento
    , mp.genero
    , mhe.cod_red
    , mhe.red
    , mhe.cod_mred
    , mhe.microred
    , mhe.provincia
    , mhe.distrito
    , mhe.cod_eess
    , nt.id_pais
    , concat(
                mp2.numero_documento
        , ' - '
        , mp2.nombres_personal
        , ' '
        , mp2.apellido_paterno_personal
        , ' '
        , mp2.apellido_materno_personal
    ) AS personal
    , concat(
                mr.numero_documento
        , ' - '
        , mr.nombres_registrador
        , ' '
        , mr.apellido_paterno_registrador
        , ' '
        , mr.apellido_materno_registrador
    ) as registrador ,
    ROW_NUMBER() OVER (PARTITION BY mp.id_tipo_documento, mp.numero_documento  ORDER BY nt.anio, nt.fecha_atencion DESC) AS rn
FROM
            maestros.nominal_trama nt
INNER JOIN maestros.nominal_trama nt2 ON nt.id_cita= nt2.id_cita
INNER JOIN maestros.maestro_paciente mp ON
            mp.id_paciente = nt.id_paciente
LEFT JOIN maestros.eess_geresa_cusco mhe ON
            mhe.id_eess = nt.id_establecimiento
LEFT JOIN maestros.maestro_his_tipo_doc mhtd ON
            mhtd.id_tipo_documento = mp.id_tipo_documento
LEFT JOIN maestros.maestro_personal mp2 ON
            mp2.id_personal = nt.id_personal
LEFT JOIN maestros.maestro_registrador mr ON
            mr.id_registrador = nt.id_registrador
LEFT JOIN maestros.maestro_his_centro_poblado mhcp ON
            nt.id_centro_poblado = mhcp.id_centro_poblado
WHERE
        (
            nt.codigo_item LIKE ANY (
                ARRAY['Z349%'
            , 'Z359%']
        )   
    )
    AND nt.anio = 2023
    AND nt2.codigo_item IN('85018','85018.01') )AS t
  WHERE t.rn=1
  
""")
    return gestante
# menores.head(2)


In [20]:
#consulta a la base de datos altitud de establecimiento 
def consulta_altitud_eess(conn):
    altitud=conn.df("SELECT cod_unico,altitud as altitud_eess FROM renaes_altitud_eess")
    altitud = altitud.rename(columns={'cod_unico': 'cod_eess'})
    return altitud
#consulta a la base de datos nombre y apellido de los menores 
def consulta_covid(cpaciente):
    nombres=cpaciente.df("SELECT * FROM paciente()")
    nombres = nombres.rename(columns={'num_doc': 'numero_documento'})
    return nombres
#consulta a la base de datos sobre las observaciones
def consulta_observaciones(conn):
    obs=conn.df("SELECT * FROM observaciones_his oh")   
    obs = obs.rename(columns={'id': 'id_obs','descricion': 'diagnostico'})
    return   obs

<h2>Crea DF Gestastes</h2> 

In [27]:
def estructura_df(gestantes,altitud):
    gestantes = gestantes.drop(['rn'], axis=1)
    gestantes['fecha_atencion'] = pd.to_datetime(gestantes['fecha_atencion'],format='%Y-%m-%d')   
    gestantes = gestantes.sort_values(by=['fecha_atencion'], inplace=False)
    gestantes = gestantes.merge(altitud, how='inner', on=['cod_eess'])
    # Reemplazar por = 0 los registros enblanco
    gestantes['hemoglobina'] = gestantes['hemoglobina'].fillna(0)
    gestantes['altitud_centro_poblado'] = gestantes['altitud_centro_poblado'].fillna(0)
    gestantes['id_paciente'] = gestantes['id_paciente'].astype('int64', errors='raise') 
    # INDEXAR
    gestantes = gestantes.set_index(['abrev_tipo_doc','numero_documento'])
    # extraer el valosr maximo
    gestantes = gestantes.groupby(gestantes.index).tail(1)
    #menor.head(2)
    gestantes=gestantes.reset_index() 
    return gestantes


<h2>Funciones </h2>

In [22]:
def fun_registros_obs(df):
    HemObs = df['hemoglobina']
    altitud = df['altitud_centro_poblado']
    conditions = [
        (HemObs == 0),
        (altitud == 0),
        (HemObs < 4.0) | (HemObs > 18.5)
        
    ]  
    choices = [1,6, 2]
    df['id_obs'] = np.select(conditions, choices, default=0)
    return df

def fun_Hemoglobina(df):
    HemObs = df['hemoglobina']
    altitud = df['altitud_eess']
    df['hemo_ajustada'] =round( HemObs - (altitud* 3.3 / 1000) * (-0.032 + (0.022 * altitud * 3.3 / 1000)),2)
    return df

def fun_diagnostico(df):    
    obs=df[(df['id_obs'] == 0) | (df['id_obs'] == 6)]
    HemoAjustado = obs['hemo_ajustada']
    conditions = [
        (HemoAjustado < 7),
        (HemoAjustado >= 7) & (HemoAjustado < 10),
        (HemoAjustado >= 10) & (HemoAjustado < 11),
        (HemoAjustado >= 11)
        ]
    choices = ['Anemia Severa', 'Anemia Moderada', 'Anemia Leve', 'Normal']
    obs['diagnostico'] = np.select(conditions, choices, default='Desconocido')
    return obs



def fun_df_observados(Estructura, obs):
    # Filtrar las filas con 'id_obs' mayor que 0
    df_observados = Estructura[Estructura['id_obs'] > 0].copy()
    # Eliminar la columna 'diagnostico'
    df_observados = df_observados.merge(obs, how='inner', on=['id_obs'])    

    return df_observados

def fun_df_anemia(Estructura):
    df_observados=Estructura[(Estructura['id_obs']==0)|(Estructura['id_obs']==6)]
  #  df_observados['edad_mes']=df_observados['edad_mes'].astype(int) 
 # df_observados.to_excel('anemia.xlsx', index=False)
    return df_observados



<h2>Calcular Anemia </h2>

In [33]:
gestante = consulta_sql(conn)

obs = consulta_observaciones(conn)
#print(obs)
#pacientes_covid = consulta_covid(cpaciente)
altitud = consulta_altitud_eess(conn)
Estructura = estructura_df(gestante, altitud)
Estructura=fun_Hemoglobina(Estructura)
Estructura=fun_registros_obs(Estructura)

observados=fun_df_observados(Estructura,obs)

anemia=fun_diagnostico(Estructura)

#anemia=fun_df_anemia(Estructura)
anemia.head(3)

d:\Irvin\Irvin\Python\Python-GERESA\modulos\gestante\../..\clases\bd\conexion2.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
C:\Users\IRVIN\AppData\Local\Temp\ipykernel_19480\1230933035.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs['diagnostico'] = np.select(conditions, choices, default='Desconocido')


,abrev_tipo_doc,numero_documento,id_paciente,anio,mes,id_establecimiento,nombre_eess,edad_anio,edad_mes,edad_dia,fecha_atencion,lote,num_pag,num_reg,codigo_item,hemoglobina,fecha_resultado_hb,id_centro_poblado,descripcion_centro_poblado,altitud_centro_poblado,talla,peso,fecha_ultima_regla,peso_pregestacional,fecha_nacimiento,genero,cod_red,red,cod_mred,microred,provincia,distrito,cod_eess,id_pais,personal,registrador,altitud_eess,hemo_ajustada,id_obs,diagnostico
0,DNI,70612612,464488532317,2023,1,2317,ACOPIA,28,338.2,10295,2023-01-01,C02,4,2,85018,12.3,2023-01-01,NaN,None,0.0,145.0,66.0,None,NaN,1994-10-25,F,1,CUSCO SUR,13,ACOMAYO,ACOMAYO,ACOPIA,2318,PER,02392295 - LUCIA GUILLERMINA BUSTINZA PACHECO,46664622 - WILLIAMS QUISPE QUISPE,3715.0,9.39,6,Anemia Moderada
1,DNI,48029432,245114432317,2023,1,2317,ACOPIA,29,349.8,10648,2023-01-25,C10,15,2,85018,15.0,2023-01-25,NaN,None,0.0,151.0,45.0,2022-12-30,NaN,1993-11-30,F,1,CUSCO SUR,13,ACOMAYO,ACOMAYO,ACOPIA,2318,PER,02392295 - LUCIA GUILLERMINA BUSTINZA PACHECO,46664622 - WILLIAMS QUISPE QUISPE,3715.0,12.09,6,Normal
5,DNI,71644239,278511972317,2023,2,2317,ACOPIA,27,330.7,10065,2023-02-20,A01,13,1,85018,11.5,2023-02-20,NaN,None,0.0,145.6,67.0,None,NaN,1995-08-01,F,1,CUSCO SUR,13,ACOMAYO,ACOMAYO,ACOPIA,2318,PER,02392295 - LUCIA GUILLERMINA BUSTINZA PACHECO,46664622 - WILLIAMS QUISPE QUISPE,3715.0,8.59,6,Anemia Moderada


<h1>Carga BD</h1>

In [32]:
conn.insert_df(anemia,'anemia','materno')
conn.insert_df(observados,'observaciones_his','materno')# registro de observados 

DataFrame insertado en la tabla anemia
DataFrame insertado en la tabla observaciones_his


<h2>Exportar al excel</h2>

In [69]:
# Obtener la fecha actual
fecha_actual = datetime.datetime.now()
nombre_mes = {
    1: "Ene", 2: "Feb", 3: "Mar", 4: "Abr",
    5: "May", 6: "Jun", 7: "Jul", 8: "Ago",
    9: "Sep", 10: "Oct", 11: "Nov", 12: "Dic"
}
# Obtener el nombre abreviado del mes en español
nombre_mes_abreviado = nombre_mes[fecha_actual.month]

# Formatear la fecha con el nombre abreviado del mes en español
fecha_formateada = fecha_actual.strftime(f"{nombre_mes_abreviado}_%d_%Y")

#observados.to_excel(f"Obs_Anemia_Gestante_Ene_{fecha_formateada}.xlsx", index=False)
anemia.to_excel(f"Anemia_Gestante_Ene_{fecha_formateada}.xlsx", index=False)